## Introduction to Python SQL Libraries

In this activity we will work with three relational databases, SQlite, MySQL and PostgreSQL. All three of these database products are used for product develop and as well as Data Science.

By default SQLite should already be installed, as part of the Python installation process. SQLite databases are serverless and self-contained, since they read and write data to a file. This means that, unlike with MySQL and PostgreSQL, you don’t even need to install and run an SQLite server to perform database operations!

To install MySQL refer to this site:

https://dev.mysql.com

To install PostgreSQL refer to this site.

https://postgresql.org

We will start off by review the following high level database schema.

![image1.png](images/image1.png)


Both ``users`` and ``posts`` will have a one-to-many relationship since one user can like many posts. Similarly, one user can post many comments, and one post can have multiple comments. So, both ``users`` and ``posts`` will also have  one-to-many releationshops with the ``comments`` table. This also applies to the ``likes`` table, so both ``users`` and ``posts`` will have a one-to-many relationship with the ``likes`` table.


## Connecting to the three databases

We will first start off by walking through creating a connection to a SQLite database, then followed by MySQL and finally PostgreSQL

### Connecting to an SQLite database using sqlite3

We will import the sqlite3 library to connect to an SQLite database in Python:

In [31]:
import sqlite3

from sqlite3 import Error


def create_connection(path):

    connection = None

    try:

        connection = sqlite3.connect(path)

        print("Connection to SQLite DB successful")

    except Error as e:

        print(f"The error '{e}' occurred")


    return connection



- Lines 1 and 2 import ``sqlite3`` and the module’s ``Error`` class.

- Line 4 defines a function ``.create_connection()`` that accepts the path to the SQLite database.

- Line 7 uses ``.connect()`` from the sqlite3 module and takes the SQLite database path as a parameter. If the database exists at the specified location, then a connection to the database is established. Otherwise, a new database is created at the specified location, and a connection is established.

- Line 8 prints the status of the successful database connection.

- Line 9 catches any ``exception`` that might be thrown if ``.connect()`` fails to establish a connection.

- Line 10 displays the error message in the console.

Now that we have our function created for ``.create_conneciton()`` lets create a database called ``my_app.sqlite`` in the default folder for this Jupyter Notebook. Note if you are using windows the syntax  would be ``"E:\\my_app.sqlite"``.


In [32]:
connection = create_connection("my_app.sqlite")

Connection to SQLite DB successful


### Connecting to a MySQL

Unlike SQLite, there’s no default Python SQL module that you can use to connect to a MySQL database. Instead, you’ll need to install a Python SQL driver for MySQL in order to interact with a MySQL database from within a Python application. One such driver is mysql-connector-python. You can download this Python SQL module with ``pip install mysql-connector-python``. You will need to open up a terminal window or command prompt depending Operating System.

MySQL is a server-based database management system. One MySQL server can have multiple databases. Unlike SQLite, where creating a connection is tantamount to creating a database, a MySQL database has a two-step process for database creation:

- Make a connection to a MySQL server.
- Execute a separate query to create the database.

Lets start off by defining a function that connects to the MySQL database server and returns the connection object:

NOTE: if you encounter an access error establishing a connection to the database using Linux use the following to resolve your error: https://stackoverflow.com/questions/39281594/error-1698-28000-access-denied-for-user-rootlocalhost

In [5]:
import mysql.connector

from mysql.connector import Error


def create_connection(host_name, user_name, user_password):

    connection = None

    try:

        connection = mysql.connector.connect(

            host=host_name,

            user=user_name,

            passwd=user_password

        )

        print("Connection to MySQL DB successful")

    except Error as e:

        print(f"The error '{e}' occurred")


    return connection


connection = create_connection("localhost", "root", "admin")

Connection to MySQL DB successful


In the above script, we define a function ``create_connection()`` that accepts three parameters:

- ``host_name``
- ``user_name``
- ``user_password``

The mysql.connector Python SQL module contains a method ``.connect()`` that you use in line 7 to connect to a MySQL database server. Once the connection is established, the connection object is returned to the calling function. Finally, in line 18 you call ``create_connection()`` with the ``host name``, ``username``, and ``password``.

So far, you’ve only established the connection. The database is not yet created. To do this, you’ll define another function ``create_database()`` that accepts two parameters:

- connection is the connection object to the database server that you want to interact with.
- query is the query that creates the database.

Here’s what this function looks like:

In [6]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")


To execute queries, you use the ``cursor`` object. The query to be executed is passed to ``cursor.execute()`` in string format.

Create a database named my_app for your social media app in the MySQL database server:

In [7]:
create_database_query = "CREATE DATABASE my_app"
create_database(connection, create_database_query)

Database created successfully


Now you’ve created a database my_app on the database server. However, the connection object returned by the ``create_connection()`` is connected to the MySQL database server. You need to connect to the my_app database. To do so, you can modify ``create_connection()`` as follows:

In [44]:
import mysql.connector

from mysql.connector import Error


def create_connection(host_name, user_name, user_password, db_name):

    connection = None

    try:

        connection = mysql.connector.connect(

            host=host_name,

            user=user_name,

            passwd=user_password,
            
            database=db_name

        )

        print("Connection to MYSQL DB successful")

    except Error as e:

        print(f"The error '{e}' occurred")


    return connection


connection = create_connection("localhost", "root", "admin", "my_app")


Connection to MYSQL DB successful


## Creating Tables

In the previous section, you saw how to connect to SQLite, MySQL, and PostgreSQL database servers using different Python SQL libraries. You created the ``my_app`` database on all three database servers. In this section, you’ll see how to ``create tables`` inside these three databases.

As discussed earlier, you’ll create four tables:

- users
- posts
- comments
- likes

we will start off with SQLite.

### SQLlite Table Creation

To execute queries in SQLite, use ``cursor.execute()``. In this section, you’ll define a function ``execute_query()`` that uses this method. Your function will accept the connection object and a query string, which you’ll pass to ``cursor.execute()``.

``.execute()`` can execute any query passed to it in the form of string. You’ll use this method to create tables in this section. In the upcoming sections, you’ll use this same method to execute ``update`` and ``delete`` queries as well. This code tries to execute the given query and prints an error message if necessary.

In [18]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")


Next we will generate our query for the ``Users`` table.

In [19]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  age INTEGER,
  gender TEXT,
  nationality TEXT
);
"""


This says to create a table ``users`` with the following five columns:

- id
- name
- age
- gender
- nationality

Finally, we will call ``execute_query()`` to create the ``users`` table. You’ll pass in the connection object that you created in the previous section, along with the ``create_users_table`` string that contains the create table query:

In [20]:
execute_query(connection, create_users_table) 

Query executed successfully


Next we will generate our query for the ``posts`` table.

In [21]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""


In [22]:
execute_query(connection, create_posts_table) 

Query executed successfully


Next we will create the quesry for both the ``comments`` and ``likes`` table

In [23]:
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

execute_query(connection, create_comments_table)  
execute_query(connection, create_likes_table)  

Query executed successfully
Query executed successfully


Now that our tables are created for SQLite lets create the tables for MySQL.

### MySQL Table Creation

We will use the ``mysql-connector-python`` Python SQL module to create tables in MySQL. Just like with SQLite, you need to pass your query to ``cursor.execute()``, which is returned by calling ``.cursor()`` on the connection object. You can create another function ``execute_query()`` that accepts the connection and query string:

In [26]:
def execute_query(connection, query):

    cursor = connection.cursor()

    try:

        cursor.execute(query) # pass our Query through

        connection.commit()

        print("Query executed successfully")

    except Error as e:

        print(f"The error '{e}' occurred")

Now lets create the ``users`` table.

In [27]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INT AUTO_INCREMENT, 
  name TEXT NOT NULL, 
  age INT, 
  gender TEXT, 
  nationality TEXT, 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""

execute_query(connection, create_users_table)


Query executed successfully


Now Lets create the ``posts`` table.

In [28]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts (
  id INT AUTO_INCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY fk_user_id (user_id) REFERENCES users(id), 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""

execute_query(connection, create_posts_table)


Query executed successfully


In [29]:
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INT AUTO_INCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL,  
  FOREIGN KEY fk_user_id (user_id) REFERENCES users(id), FOREIGN KEY (post_id) REFERENCES posts (id), 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""

create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INT AUTO_INCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY fk_user_id (user_id) REFERENCES users(id), FOREIGN KEY (post_id) REFERENCES posts (id), 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""

execute_query(connection, create_comments_table)
execute_query(connection, create_likes_table)

Query executed successfully
Query executed successfully


### Inserting Records using SQLite

Now that we have created our tables its times to add some data to them. We will start by adding five records into the ``users`` table.

In [33]:
create_users = """
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

execute_query(connection, create_users)  

Query executed successfully


Since you set the id column to auto-increment, you don’t need to specify the value of the id column for these users. The users table will auto-populate these five records with id values from 1 to 5.

We will now insert six records into the ``posts`` table.


In [34]:
create_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""

execute_query(connection, create_posts)  

Query executed successfully


It’s important to mention that the ``user_id`` column of the posts table is a foreign key that references the ``id`` column of the ``users`` table. This means that the ``user_id`` column must contain a value that already exists in the ``id`` column of the ``users`` table. If it doesn’t exist, then you’ll see an error.

Now lets inserts records into the ``comments`` and ``likes`` tables:

In [35]:
create_comments = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

create_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

execute_query(connection, create_comments)
execute_query(connection, create_likes)  

Query executed successfully
Query executed successfully


### Inserting Records using MySQL

There are two ways to insert records into MySQL databases from a Python application. The first approach is similar to SQLite. You can store the ``INSERT INTO`` query in a string and then use ``cursor.execute()`` to insert records.

Earlier, you defined a wrapper function ``execute_query()`` that you used to insert records. You can use this same function now to insert records into your MySQL table. 

The following script inserts records into the ``users`` table using ``execute_query()``:

In [45]:
create_users = """
INSERT INTO
  `users` (`name`, `age`, `gender`, `nationality`)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

execute_query(connection, create_users) 

Query executed successfully


The second approach uses ``cursor.executemany()``, which accepts two parameters:

- The query string containing placeholders for the records to be inserted
- The list of records that you want to insert

Look at the following example, which inserts two records into the ``likes`` table:

In [46]:
sql = "INSERT INTO likes ( user_id, post_id ) VALUES ( %s, %s )"
val = [(4, 5), (3, 4)]

cursor = connection.cursor()
cursor.executemany(sql, val)
connection.commit()


IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`my_app`.`likes`, CONSTRAINT `likes_ibfk_2` FOREIGN KEY (`post_id`) REFERENCES `posts` (`id`))

Go ahead and make use of the existing data to complete adding records to the tables

## Selecting records

In this section, you’ll see how to select records from database tables using the different Python SQL modules. In particular, you’ll see how to perform ``SELECT`` queries on your SQLite and MySQL.


### Selecting records using SQLite

To select records using SQLite, you can again use ``cursor.execute()``. However, after you’ve done this, you’ll need to call ``.fetchall()``. This method returns a list of tuples where each tuple is mapped to the corresponding row in the retrieved records.

To simplify the process, you can create a function ``execute_read_query()``:

In [37]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")


We will start off be creating a query for selecting all ``users`` from the ``users`` table. To do this we perform a ``SELECT`` query with the ``*`` which denotes all records, with a very large database this

In [40]:
select_users = "SELECT * from users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)


(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


As you can see each record we addhas been given an auto-numbered private key starting with ``1``.

Lets now run this with the ``posts`` table.

In [41]:
select_posts = "SELECT * FROM posts"
posts = execute_read_query(connection, select_posts)

for post in posts:
    print(post)


(1, 'Happy', 'I am feeling very happy today', 1)
(2, 'Hot Weather', 'The weather is very hot today', 2)
(3, 'Help', 'I need some help with my work', 2)
(4, 'Great News', 'I am getting married', 1)
(5, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(6, 'Party', 'Anyone up for a late-night party today?', 3)


## JOIN

We will now make use of the ``JOIN`` to join data from both the ``posts`` table and the ``users`` table. We will use the``SELECT`` query for ``users.id, users.name, posts.description``

In [42]:
select_users_posts = """
SELECT
  users.id,
  users.name,
  posts.description
FROM
  posts
  INNER JOIN users ON users.id = posts.user_id
"""

users_posts = execute_read_query(connection, select_users_posts)

for users_post in users_posts:
    print(users_post)


(1, 'James', 'I am feeling very happy today')
(2, 'Leila', 'The weather is very hot today')
(2, 'Leila', 'I need some help with my work')
(1, 'James', 'I am getting married')
(5, 'Elizabeth', 'It was a fantastic game of tennis')
(3, 'Brigitte', 'Anyone up for a late-night party today?')


## Multiple Join

We can also select data from three related tables by implementing multiple ``JOIN`` operators.

In [8]:
select_posts_comments_users = """
SELECT
  posts.description as post,
  text as comment,
  name
FROM
  posts
  INNER JOIN comments ON posts.id = comments.post_id
  INNER JOIN users ON users.id = comments.user_id
"""

posts_comments_users = execute_read_query(
    connection, select_posts_comments_users
)

for posts_comments_user in posts_comments_users:
    print(posts_comments_user)


('Anyone up for a late-night party today?', 'Count me in', 'James')
('I need some help with my work', 'What sort of help?', 'Elizabeth')
('I am getting married', 'Congrats buddy', 'Leila')
('It was a fantastic game of tennis', 'I was rooting for Nadal though', 'Mike')
('I need some help with my work', 'Help with your thesis?', 'Leila')
('I am getting married', 'Many congratulations', 'Elizabeth')


## Where

Now we will execute a ``SELECT`` query that returns the post, along with the total number of likes that the post received:

In [43]:
select_post_likes = """
SELECT
  description as Post,
  COUNT(likes.id) as Likes
FROM
  likes,
  posts
WHERE
  posts.id = likes.post_id
GROUP BY
  likes.post_id
"""

post_likes = execute_read_query(connection, select_post_likes)

for post_like in post_likes:
    print(post_like)


('The weather is very hot today', 1)
('I need some help with my work', 1)
('I am getting married', 2)
('It was a fantastic game of tennis', 1)
('Anyone up for a late-night party today?', 2)


## Selecting records with MySQL

Selecting records with MySQL is the same as SQLite

In [51]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")


In [52]:
select_users = "SELECT * FROM users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)


(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


## Pandas


In [47]:
data = {
    'CHN': {'COUNTRY': 'China', 'POP': 1_398.72, 'AREA': 9_596.96,
            'GDP': 12_234.78, 'CONT': 'Asia'},
    'IND': {'COUNTRY': 'India', 'POP': 1_351.16, 'AREA': 3_287.26,
            'GDP': 2_575.67, 'CONT': 'Asia', 'IND_DAY': '1947-08-15'},
    'USA': {'COUNTRY': 'US', 'POP': 329.74, 'AREA': 9_833.52,
            'GDP': 19_485.39, 'CONT': 'N.America',
            'IND_DAY': '1776-07-04'},
    'IDN': {'COUNTRY': 'Indonesia', 'POP': 268.07, 'AREA': 1_910.93,
            'GDP': 1_015.54, 'CONT': 'Asia', 'IND_DAY': '1945-08-17'},
    'BRA': {'COUNTRY': 'Brazil', 'POP': 210.32, 'AREA': 8_515.77,
            'GDP': 2_055.51, 'CONT': 'S.America', 'IND_DAY': '1822-09-07'},
    'PAK': {'COUNTRY': 'Pakistan', 'POP': 205.71, 'AREA': 881.91,
            'GDP': 302.14, 'CONT': 'Asia', 'IND_DAY': '1947-08-14'},
    'NGA': {'COUNTRY': 'Nigeria', 'POP': 200.96, 'AREA': 923.77,
            'GDP': 375.77, 'CONT': 'Africa', 'IND_DAY': '1960-10-01'},
    'BGD': {'COUNTRY': 'Bangladesh', 'POP': 167.09, 'AREA': 147.57,
            'GDP': 245.63, 'CONT': 'Asia', 'IND_DAY': '1971-03-26'},
    'RUS': {'COUNTRY': 'Russia', 'POP': 146.79, 'AREA': 17_098.25,
            'GDP': 1_530.75, 'IND_DAY': '1992-06-12'},
    'MEX': {'COUNTRY': 'Mexico', 'POP': 126.58, 'AREA': 1_964.38,
            'GDP': 1_158.23, 'CONT': 'N.America', 'IND_DAY': '1810-09-16'},
    'JPN': {'COUNTRY': 'Japan', 'POP': 126.22, 'AREA': 377.97,
            'GDP': 4_872.42, 'CONT': 'Asia'},
    'DEU': {'COUNTRY': 'Germany', 'POP': 83.02, 'AREA': 357.11,
            'GDP': 3_693.20, 'CONT': 'Europe'},
    'FRA': {'COUNTRY': 'France', 'POP': 67.02, 'AREA': 640.68,
            'GDP': 2_582.49, 'CONT': 'Europe', 'IND_DAY': '1789-07-14'},
    'GBR': {'COUNTRY': 'UK', 'POP': 66.44, 'AREA': 242.50,
            'GDP': 2_631.23, 'CONT': 'Europe'},
    'ITA': {'COUNTRY': 'Italy', 'POP': 60.36, 'AREA': 301.34,
            'GDP': 1_943.84, 'CONT': 'Europe'},
    'ARG': {'COUNTRY': 'Argentina', 'POP': 44.94, 'AREA': 2_780.40,
            'GDP': 637.49, 'CONT': 'S.America', 'IND_DAY': '1816-07-09'},
    'DZA': {'COUNTRY': 'Algeria', 'POP': 43.38, 'AREA': 2_381.74,
            'GDP': 167.56, 'CONT': 'Africa', 'IND_DAY': '1962-07-05'},
    'CAN': {'COUNTRY': 'Canada', 'POP': 37.59, 'AREA': 9_984.67,
            'GDP': 1_647.12, 'CONT': 'N.America', 'IND_DAY': '1867-07-01'},
    'AUS': {'COUNTRY': 'Australia', 'POP': 25.47, 'AREA': 7_692.02,
            'GDP': 1_408.68, 'CONT': 'Oceania'},
    'KAZ': {'COUNTRY': 'Kazakhstan', 'POP': 18.53, 'AREA': 2_724.90,
            'GDP': 159.41, 'CONT': 'Asia', 'IND_DAY': '1991-12-16'}
}

columns = ('COUNTRY', 'POP', 'AREA', 'GDP', 'CONT', 'IND_DAY')


In [48]:
import pandas as pd

In [49]:
df = pd.DataFrame(data=data).T
df

,COUNTRY,POP,AREA,GDP,CONT,IND_DAY
CHN,China,1398.72,9596.96,12234.8,Asia,NaN
IND,India,1351.16,3287.26,2575.67,Asia,1947-08-15
USA,US,329.74,9833.52,19485.4,N.America,1776-07-04
IDN,Indonesia,268.07,1910.93,1015.54,Asia,1945-08-17
BRA,Brazil,210.32,8515.77,2055.51,S.America,1822-09-07
PAK,Pakistan,205.71,881.91,302.14,Asia,1947-08-14
NGA,Nigeria,200.96,923.77,375.77,Africa,1960-10-01
BGD,Bangladesh,167.09,147.57,245.63,Asia,1971-03-26
RUS,Russia,146.79,17098.2,1530.75,NaN,1992-06-12
MEX,Mexico,126.58,1964.38,1158.23,N.America,1810-09-16


Writing to a CSV file

In [50]:
df.to_csv('new_data.csv')

Reading from a CSV file

In [53]:
df = pd.read_csv('new_data.csv', index_col=0)
df

,COUNTRY,POP,AREA,GDP,CONT,IND_DAY
CHN,China,1398.72,9596.96,12234.78,Asia,NaN
IND,India,1351.16,3287.26,2575.67,Asia,1947-08-15
USA,US,329.74,9833.52,19485.39,N.America,1776-07-04
IDN,Indonesia,268.07,1910.93,1015.54,Asia,1945-08-17
BRA,Brazil,210.32,8515.77,2055.51,S.America,1822-09-07
PAK,Pakistan,205.71,881.91,302.14,Asia,1947-08-14
NGA,Nigeria,200.96,923.77,375.77,Africa,1960-10-01
BGD,Bangladesh,167.09,147.57,245.63,Asia,1971-03-26
RUS,Russia,146.79,17098.25,1530.75,NaN,1992-06-12
MEX,Mexico,126.58,1964.38,1158.23,N.America,1810-09-16


## Using Pandas to Write and Read Excel Files

Microsoft Excel is probably the most widely-used spreadsheet software. While older versions used binary .xls files, Excel 2007 introduced the new XML-based .xlsx file. You can read and write Excel files in Pandas, similar to CSV files. However, you’ll need to install the following Python packages first:

- xlwt to write to .xls files
- openpyxl or XlsxWriter to write to .xlsx files
- xlrd to read Excel files

``pip install xlwt openpyxl xlsxwriter xlrd``

Note that you don’t have to install all these packages. For example, you don’t need both ``openpyxl`` and ``XlsxWriter``. If you’re going to work just with .xls files, then you don’t need any of them! However, if you intend to work only with .xlsx files, then you’re going to need at least one of them, but not ``xlwt``. Take some time to decide which packages are right for your project.

## Writing to and Excel file format

Writing an .xlsx is straightforward using the ``.to_excel()`` method.

In [54]:
df.to_excel('Istvan_data.xlsx')

### Read an Excel File

You can load data from Excel files with the ``read_excel()`` method:

In [55]:
import pandas as pd
import openpyxl
#df = pd.read_excel('data.xlsx', index_col=0)
#print(df)

ff=pd.read_excel("Istvan_data.xlsx", engine='openpyxl')
print(ff)

   Unnamed: 0     COUNTRY      POP      AREA       GDP       CONT     IND_DAY
0         CHN       China  1398.72   9596.96  12234.78       Asia         NaN
1         IND       India  1351.16   3287.26   2575.67       Asia  1947-08-15
2         USA          US   329.74   9833.52  19485.39  N.America  1776-07-04
3         IDN   Indonesia   268.07   1910.93   1015.54       Asia  1945-08-17
4         BRA      Brazil   210.32   8515.77   2055.51  S.America  1822-09-07
5         PAK    Pakistan   205.71    881.91    302.14       Asia  1947-08-14
6         NGA     Nigeria   200.96    923.77    375.77     Africa  1960-10-01
7         BGD  Bangladesh   167.09    147.57    245.63       Asia  1971-03-26
8         RUS      Russia   146.79  17098.25   1530.75        NaN  1992-06-12
9         MEX      Mexico   126.58   1964.38   1158.23  N.America  1810-09-16
10        JPN       Japan   126.22    377.97   4872.42       Asia         NaN
11        DEU     Germany    83.02    357.11   3693.20     Europ

## Writing Pandas dataframe to an SQLite database

If you do not have it installed install ``pip install sqlalchemy``.

We will start of by importing the ``create_engine`` from ``sqlalchemy``. We then will create our database engine.

The syntax format for the ``create_engine()``:

**Unix/Mac** - 3 initial slashes in total
engine = create_engine('sqlite:///relative_path/foo.db')

**Unix/Mac** - 4 initial slashes in total
engine = create_engine('sqlite:////absolute/path/to/foo.db')

**Windows**
engine = create_engine('sqlite:///C:\\path\\to\\foo.db')

We will create a new database called ``new_data.db``.


In [56]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///istvan_data.db', echo=False)

Next we will setup our datatypes for the data that will be uploaded to our database

In [58]:
dtypes = {'POP': 'float64', 'AREA': 'float64', 'GDP': 'float64', 'IND_DAY': 'datetime64'}
df = pd.DataFrame(data=data).T.astype(dtype=dtypes)
df.dtypes

COUNTRY            object
POP               float64
AREA              float64
GDP               float64
CONT               object
IND_DAY    datetime64[ns]
dtype: object

Next we will right our dataframe to our SQLite database using the ``.to_sql()`` method.

In [59]:
df.to_sql('istvan_data.db', con=engine, index_label='ID')

To read from this database we just call the ``.read_sql()`` method.

In [61]:
df = pd.read_sql('istvan_data.db', con=engine, index_col='ID')
df

,COUNTRY,POP,AREA,GDP,CONT,IND_DAY
ID,,,,,,
CHN,China,1398.72,9596.96,12234.78,Asia,NaT
IND,India,1351.16,3287.26,2575.67,Asia,1947-08-15
USA,US,329.74,9833.52,19485.39,N.America,1776-07-04
IDN,Indonesia,268.07,1910.93,1015.54,Asia,1945-08-17
BRA,Brazil,210.32,8515.77,2055.51,S.America,1822-09-07
PAK,Pakistan,205.71,881.91,302.14,Asia,1947-08-14
NGA,Nigeria,200.96,923.77,375.77,Africa,1960-10-01
BGD,Bangladesh,167.09,147.57,245.63,Asia,1971-03-26
RUS,Russia,146.79,17098.25,1530.75,None,1992-06-12
